In [8]:
import pandas as pd
import numpy as np

In [2]:
# Define number of events in data
number_of_events_ = 100

## Load the DataFrames Stored during Data Preprocessing

In [3]:
'''Refer to Data Preprocessing Notebook to Understand the Semantics and Column Headers'''

'Refer to Data Preprocessing Notebook to Understand the Semantics and Column Headers'

In [4]:
track_global_df_ = pd.read_msgpack('data/track_global_df_.msgpack')
track_param_global_df_ = pd.read_msgpack('data/track_param_global_df_.msgpack')
rechit_global_df_ = pd.read_msgpack('data/rechit_global_df_.msgpack')
rechit_param_global_df_ = pd.read_msgpack('data/rechit_param_global_df_.msgpack')

In [5]:
track_count_ = pd.DataFrame.to_dict(pd.read_csv('data/track_count_.csv'))
rechit_count_ = pd.DataFrame.to_dict(pd.read_csv('data/rechit_count_.csv'))
track_ids_ = pd.DataFrame.to_dict(pd.read_csv('data/track_ids_.csv'))
rechit_ids_ = pd.DataFrame.to_dict(pd.read_csv('data/rechit_ids_.csv'))

## Create the Adjacency Matrices

In [6]:
'''
Graphs are created on a per-event basis. We build an adjacency matrix of rechits for each individual event.
The rechit connections are defined by the track that they belong to.
The target label for each node is taken as the first tp_index in its rechit_tp_list

#TODO: Incorporate a more flexible labeling schema - can you label the edges using the 'extra' tp index?
Can this learn more interesting structures for the graph(s)?
# Solution: Use the same node in different graphs - same as above but implementation-wise easier to do.

#TODO: What information do we use to weight the edges in the graph?
Differences in rechit parameters?
Rechit vs. Track Parameters?
'''

"\nGraphs are created on a per-event basis. We build an adjacency matrix of rechits for each individual event.\nThe rechit connections are defined by the track that they belong to.\nThe target label for each node is taken as the first tp_index in its rechit_tp_list\n\n#TODO: Incorporate a more flexible labeling schema - can you label the edges using the 'extra' tp index?\nCan this learn more interesting structures for the graph(s)?\n# Solution: Use the same node in different graphs - same as above but implementation-wise easier to do.\n\n#TODO: What information do we use to weight the edges in the graph?\nDifferences in rechit parameters?\nRechit vs. Track Parameters?\n"

In [9]:
uncut_rechit_ids_ = np.load('data/uncut_rechit_ids_.npy', encoding='bytes')

FileNotFoundError: [Errno 2] No such file or directory: 'data/uncut_rechit_ids_.npy'

In [ ]:
import numpy as np
'''TODO: Convert data to string instead of bytes and check overheads'''



edge_src_ = []
edge_dest_ = []

x = []

for event_id_ in range(number_of_events_):
    track_event_df_ = track_global_df_[track_global_df_[b'event_id'] == event_id_]
    number_of_rechits_in_event_ = len(rechit_global_df_[rechit_global_df_[b'event_id']==event_id_])
    
    for track_rechit_id_array_ in track_event_df_[b'rechit_ids']:
        uncut_rechits_ = [i for i in track_rechit_id_array_ if i in rechit_ids_['track_matched']]

In [ ]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNLayer, self).__init__()
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3-5: Start propagating messages with "add" aggregation.
        return self.propagate('add', edge_index, x=x, num_nodes=x.size(0))

    def message(self, x_j, edge_index, num_nodes):
        # x_j has shape [E, out_channels]

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, num_nodes, dtype=x_j.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

In [ ]:
from torch_geometric.data import Data

# Sample Edge Label Definition for Rechits - Adjacency List?
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [ ]:
'''Define the 2-layer GCN'''

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, data.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)


In [ ]:
# CUDA available on cmg-gpu1080
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()